## Objetivos de aprendizagem

- Entender o que é web scraping e quando usar.
- Conhecer o fluxo de trabalho para fazer web scraping.
- Conhecer os principais tipos de problemas que podemos encontrar.
- Conhecer as principais ferramentas para fazer web scraping.
- Conhecer a política do web scraping.

## Por que web scraping?

**Tudo o que você vê na internet pode se transformar dados para analisar!**

### No Direito

Os dados do judiciário são **públicos**, mas não são **abertos**.

Existem novas iniciativas para tornar os dados do judiciário mais abertos, como o [DataJud](https://www.cnj.jus.br/datajud), mas ainda há muito a ser feito.


## Tipos de problemas

- **APIs disponíveis**: O site fornece uma forma estruturada e documentada para acessar as páginas.

- **APIs escondidas**: O site não fornece uma forma estruturada e documentada para acessar as páginas, mas internamente é alimentado por uma API não documentada, que podemos descobrir e usar.

- **HTML estático**: O site não fornece uma forma estruturada de acessar as páginas, e as páginas são geradas de forma estática (carregam sem necessidade de usar um navegador).

- **HTML dinâmico**: O site não fornece uma forma estruturada de acessar as páginas, e as páginas são geradas de forma dinâmica.

- ...

## Ferramentas

- [requests](https://requests.readthedocs.io/en/master/): pacote do Python para acessar sites.
- [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): pacote do Python para transformar arquivos html em páginas estruturadas.
- [Selenium](https://www.selenium.dev/): ferramenta para automatizar ações em um navegador. Útil para páginas dinâmicas.

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Exemplo: API escondida do TJSP

In [3]:
cd = '590005PV80000'

u = f'https://api.tjsp.jus.br/processo/cpopg/dadosbasicos/{cd}'

r = requests.post(
  u,
  headers = {'Content-Type': 'application/json; charset=utf-8'}
)

r.status_code
r.json()

{'codigo': '590005PV80000',
 'classe': 'Procedimento do Juizado Especial Cível',
 'assunto': 'Obrigações',
 'localFisico': '',
 'area': 'Cível',
 'distribuicao': 'Vara do Juizado Especial Cível e Criminal - Foro de Fernandópolis',
 'valor': 'R$         5.121,68',
 'virtual': 'True',
 'temCustaPendente': 'False'}

## Exemplo: API escondida do TJSP

In [4]:
r.json()

{'codigo': '590005PV80000',
 'classe': 'Procedimento do Juizado Especial Cível',
 'assunto': 'Obrigações',
 'localFisico': '',
 'area': 'Cível',
 'distribuicao': 'Vara do Juizado Especial Cível e Criminal - Foro de Fernandópolis',
 'valor': 'R$         5.121,68',
 'virtual': 'True',
 'temCustaPendente': 'False'}

# HTML

## HTML

- HTML (Hypertext Markup Language) é uma linguagem de marcação.

- Por trás de todo site há pelo menos um arquivo `.html`.

## Tags, atributos e texto

- Tags (head, body, h1, p, ...) demarcam as seções e sub-seções

- enquanto atributos (charset, style, ...) mudam como essas seções são renderizadas pelo navegador.

## Estrutura

O HTML do exemplo, na verdade, é isso aqui:

## Usando o BeautifulSoup

- Exemplo: coletando todas as tags `<p>` (parágrafos)

In [5]:
# Ler o HTML
with open("html_exemplo.html") as f:
  html = BeautifulSoup(f)

# Imprimir o HTML
print(html.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   Título da abinha do navegador
  </title>
 </head>
 <body>
  <h1>
   Título Grande
  </h1>
  <h2>
   Título um pouco menor
  </h2>
  <p>
   Sou um parágrafo!
  </p>
  <p style="color: blue;">
   Sou um parágrafo azul.
  </p>
 </body>
</html>



## Usando o BeautifulSoup

- Exemplo: coletando todas as tags `<p>` (parágrafos)

In [7]:
# Extrair o texto contido em cada um dos nodes
html.find_all('p')

[<p>Sou um parágrafo!</p>, <p style="color: blue;">Sou um parágrafo azul.</p>]

## Extraindo texto e atributos

In [8]:
# Extrair o texto contido em cada um dos nodes
html.p.text
html.find_all('p')[0].text

# Extrair os atributos
html.find_all('p')[0].attrs
html.find_all('p')[1].attrs

{'style': 'color: blue;'}

## Aplicação: Descobrindo o Código do Processo!

In [9]:
with open("cpopg.html") as f:
  html_cpopg = BeautifulSoup(f)

# tr with class='fundocinza1'
trs = html_cpopg.find_all('tr', {'class': 'fundocinza1'})

links = [tr.a for tr in trs]
codigos = [link.attrs['name'][0:13] for link in links]

## Aplicação: Iterar! {.smaller}

In [10]:
# read data and transform to dataframe
lista = []
for cd in codigos:
  u_base = f'https://api.tjsp.jus.br/processo/cpopg/dadosbasicos/{cd}'
  r = requests.post(
    u_base,
    headers = {'Content-Type': 'application/json; charset=utf-8'}
  )
  lista.append(r.json())

da = pd.DataFrame(lista)

da.head()

,codigo,classe,assunto,localFisico,area,distribuicao,valor,virtual,temCustaPendente
0,590005PV80000,Procedimento do Juizado Especial Cível,Obrigações,,Cível,Vara do Juizado Especial Cível e Criminal - Fo...,"R$ 5.121,68",True,False
1,0G00095UP0000,Procedimento do Juizado Especial Cível,Indenização por Dano Material,,Cível,2ª Vara do Juizado Especial Cível - Vergueiro ...,"R$ 11.000,00",True,False
2,2S001OAHW0000,Procedimento Comum Cível,Sustação de Protesto,,Cível,38ª Vara Cível - Foro Central Cível,"R$ 33.968,14",True,False
3,7Y0009WMF0000,Procedimento Comum Cível,Perdas e Danos,,Cível,3ª Vara Cível - Foro de Itu,"R$ 16.949,89",True,False
4,0G0009UX40000,Procedimento do Juizado Especial Cível,Indenização por Dano Moral,,Cível,Juizado Especial Cível Anexo Mackenzie - Foro ...,"R$ 16.829,26",True,False


## Política do web scraping

### Quando usar

- Quando precisamos coletar um volume grande de dados da internet

### Quando não usar

- Existem formas mais simples de obter os dados (API, base de dados, etc.)

- Os termos de uso do site não nos permitem fazer isso.

- As informações do site não são públicas.

## Cuidados

- Risco de derrubar ou comprometer a estabilidade do site.

- Vale à pena conversar com a entidade detentora dos dados.

## Resumo

- Web scraping é uma técnica para coletar dados de sites.
- Existe um fluxo para fazer web scraping.
- Existem 4 tipos principais 'níveis' de dificuldade, sendo que o primeiro deles não é considerado web scraping.
- Existem várias ferramentas para fazer web scraping. Em python, as principais são requests, beautiful soup e selenium.
- É importante conhecer a política do site para fazer web scraping.

## Curiosidade: como eu descobri essa API escondida?

Longa história, mas basicamente:

- instalei o aplicativo do TJSP no celular
- usei o aplicativo tPacketCapture para interceptar as requisições do aplicativo
- descobri que o aplicativo usa uma API escondida para buscar os processos
- descobri que a API escondida usa o mesmo código do processo que aparece na URL do site
- descobri que a API escondida retorna os dados do processo em formato JSON